In [1]:
!python --version

Python 3.7.10


In [2]:
!brew install portaudio

Updating Homebrew...
==> Auto-updated Homebrew!
Updated 1 tap (homebrew/core).
==> Updated Formulae
Updated 11 formulae.

==> Downloading https://ghcr.io/v2/homebrew/core/portaudio/manifests/19.7.0
Already downloaded: /Users/jaeyoungshin/Library/Caches/Homebrew/downloads/826f43092f6bdf1eeccd7b02660cdf10f9a58d963a5d8f99e1b6d05f3af4a27e--portaudio-19.7.0.bottle_manifest.json
==> Downloading https://ghcr.io/v2/homebrew/core/portaudio/blobs/sha256:f9ae9716
==> Downloading from https://pkg-containers-az.githubusercontent.com/ghcr1/blobs
######################################################################## 100.0%
Error: portaudio HEAD-32f5cbd is already installed
To install 19.7.0, first run:
  brew unlink portaudio
Removing: /Users/jaeyoungshin/Library/Caches/Homebrew/portaudio--19.7.0... (138.6KB)


In [3]:
!pip install pyaudio

In [5]:
!pip list

Package                 Version
----------------------- -------------------
absl-py                 0.12.0
appdirs                 1.4.4
appnope                 0.1.2
argon2-cffi             20.1.0
astunparse              1.6.3
async-generator         1.10
attrs                   20.3.0
audioread               2.1.9
backcall                0.2.0
beautifulsoup4          4.9.3
bleach                  3.3.0
cachetools              4.2.1
certifi                 2020.12.5
cffi                    1.14.5
chardet                 4.0.0
cycler                  0.10.0
decorator               5.0.7
defusedxml              0.7.1
docopt                  0.6.2
entrypoints             0.3
flatbuffers             1.12
gast                    0.3.3
google-auth             1.30.0
google-auth-oauthlib    0.4.4
google-pasta            0.2.0
grpcio                  1.32.0
h5py                    2.10.0
idna                    2.10
importlib-metadata      4.0.0
ipykernel               5.5.3
ipython          

In [8]:
!pip install numpy

  Using cached numpy-1.20.2-cp37-cp37m-macosx_10_9_x86_64.whl (16.0 MB)


In [10]:
!pip install scipy

  Using cached scipy-1.6.2-cp37-cp37m-macosx_10_9_x86_64.whl (30.7 MB)


In [1]:
# 입출력 장치 확인
import pyaudio

audio = pyaudio.PyAudio()

for index in range(audio.get_device_count()):
    desc = audio.get_device_info_by_index(index)
    print("DEVICE: {device}, INDEX: {index}, RATE: {rate} ".format(
        device=desc["name"], index=index, rate=int(desc["defaultSampleRate"])))

DEVICE: MacBook Pro 마이크, INDEX: 0, RATE: 48000 
DEVICE: MacBook Pro 스피커, INDEX: 1, RATE: 48000 
DEVICE: iShowU Audio Capture, INDEX: 2, RATE: 44100 
DEVICE: 통합 기기, INDEX: 3, RATE: 44100 
DEVICE: 다중 출력 기기, INDEX: 4, RATE: 44100 


In [10]:
!pip install noisereduce
!pip install scipy
!pip install requests
!brew uninstall portaudio
!brew install portaudio --HEAD

Uninstalling /usr/local/Cellar/portaudio/19.7.0... (33 files, 528.4KB)
Updating Homebrew...
==> Auto-updated Homebrew!
Updated 2 taps (homebrew/core and homebrew/cask).
==> New Formulae
fabric-installer    lychee              revive              webhook
gnupg@2.2           macchina            seqkit
lefthook            mr2                 stp
==> Updated Formulae
Updated 388 formulae.
==> New Casks
arkiwi              flameshot           hook                optimus-player
chia                fpc-laz             isyncer             slidepilot
firefly             fpc-src-laz         menuwhere
==> Updated Casks
Updated 226 casks.
==> Deleted Casks
fpcsrc

==> Downloading https://ghcr.io/v2/homebrew/core/pkg-config/manifests/0.29.2_3
######################################################################## 100.0%
==> Downloading https://ghcr.io/v2/homebrew/core/pkg-config/blobs/sha256:0040b6e
==> Downloading from https://pkg-containers-az.githubusercontent.com/ghcr1/blobs
##################

In [7]:
!pip install SpeechRecognition

## 라이브러리

In [2]:
import speech_recognition as sr
from collections import defaultdict
import pyaudio
import wave
import audioop
from collections import deque
import os
import urllib
import time
import math
import numpy as np
import noisereduce as nr
from scipy.io import wavfile
import soundfile as sf

/Users/jaeyoungshin/opt/anaconda3/envs/python3.7/lib/python3.7/site-packages/noisereduce/noisereduce.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## 노이즈제거x 실시간 오디오파일 묵음기준 저장

In [ ]:
# Microphone stream config.
CHUNK = 1000
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
THRESHOLD = 1500
SILENCE_LIMIT = 1 
PREV_AUDIO = 0.5
RECORD_SECONDS = 10

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory.' +  directory)


def audio_int(num_samples=10):
    print("Getting intensity values from mic.")
    p = pyaudio.PyAudio()

    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    values = [math.sqrt(abs(audioop.avg(stream.read(CHUNK), 4))) 
              for x in range(num_samples)] 
    values = sorted(values, reverse=True)
    r = sum(values[:int(num_samples * 0.2)]) / int(num_samples * 0.2)
    print(" Finished ")
    print(" Average audio intensity is ", r)
    stream.close()
    p.terminate()
    return r


def listen_for_speech(threshold=THRESHOLD, num_phrases=-1):
    #Open stream
    p = pyaudio.PyAudio()

    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    print("* Listening mic. ")
    audio2send = []
    cur_data = '' 
    rel = RATE/CHUNK
    slid_win = deque(maxlen=int(SILENCE_LIMIT * rel))
    prev_audio = deque(maxlen=int(PREV_AUDIO * rel))
    started = False
    n = num_phrases
    response = []

    while (num_phrases == -1):
        cur_data = stream.read(CHUNK,exception_on_overflow=False)
        slid_win.append(math.sqrt(abs(audioop.avg(cur_data, 4))))
        if(sum([x > threshold for x in slid_win]) > 0):
            if(not started):
                print("Starting record of phrase")
                started = True
            audio2send.append(cur_data)
        elif (started is True):
            print("Finished")
            filename = save_speech(list(prev_audio) + audio2send, p)
            text = stt_google_wav(filename)
            started = False
            slid_win = deque(maxlen=int(SILENCE_LIMIT * rel))
            prev_audio = deque(maxlen=int(0.5 * rel)) 
            audio2send = []
           # n -= 1
            print("Listening ...")
        else:
            prev_audio.append(cur_data)

    print("* Done recording")
    stream.close()
    p.terminate()

    return response


def save_speech(data, p):
    try:
        foldername = 'original_audio/'+str(time.strftime('%Y-%m-%d'))+'/'
        createFolder(foldername)
        filename = foldername+'original_output_'+str(int(time.strftime('%H%M%S')))
        data = b''.join(data)
        wf = wave.open(filename + '.wav', 'wb')
        wf.setnchannels(1)
        wf.setsampwidth(p.get_sample_size(FORMAT))
        wf.setframerate(RATE) 
        wf.writeframes(data)
        wf.close()
        return filename + '.wav'
    except data is None:
        print('Stop Recording')
        
def stt_google_wav(audio_fname):
    AUDIO_FILE = f"{audio_fname}"
    text = []
    # audio file을 audio source로 사용
    r = sr.Recognizer()
    with sr.AudioFile(AUDIO_FILE) as source:
        audio = r.record(source)  # 전체 audio file 읽기

    # 구글 웹 음성 API로 인식 (하루에 제한 50회)
    try:
        print("Google Speech Recognition thinks you said : " + r.recognize_google(audio, language='ko'))
    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand audio")
    except sr.RequestError as e:
        print("Could not request results from Google Speech Recognition service; {0}".format(e))


if(__name__ == '__main__'):
    listen_for_speech()
    audio_int()

* Listening mic. 
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition thinks you said : 파이썬 구형 코드로 배우는
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition thinks you said : 파이썬 구현 코드로 배우는
Listening ...
Starting record of phrase
Finished
Google Speech Recognition thinks you said : 백신 문법
Listening ...
Starting record of phrase
Finished
Google Speech Recognition thinks you said : 1번 버스
Listening ...
Starting record of phrase
Finished
Google Speech Recognition thinks you said : 행복과
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition thinks you said : 핵심 문법과 다양한 실용 테크닉
Listening ...
Starting record of phrase
Finished
Google Speech Recog

Finished
Google Speech Recognition thinks you said : 잘 나오는데 그 0.5% 하니까 얘가 분석을 할 때 좀 짧은 감이 있어 가지고 오히려 너희들이 주스를 하면은 성능이 떨어지는 거 같더라고요 그래 가지고 지금 좀 SNL 브금 기준으로 했을 때 3 너를 적용을 했었는데 그게 손잡이 그 비율을 그때는 임의적으로 했었는데 그거를 좀 자세히 어떻게 구체적으로 할 것 좀 생각을 한번 찾아보겠습니다 고생 많으셨습니다 노이즈리덕션 페이지는 편해 질 덕산으로 검색하면 나와요 아네 감사합니다 고생 많으셨습니다 오늘 수고하신데 도움을 주셔서 감사하고요 해 볼까요
Listening ...
Starting record of phrase
Finished
Could not request results from Google Speech Recognition service; recognition connection failed: [Errno 32] Broken pipe
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not unders

Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record

Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand 

Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Could not request results from Google Speech Recognition service; recognition connection failed: [Errno 32] Broken pipe
Listening ...
Starting record of phrase
Finished
Google Speech R

Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starti

Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starti

Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starting record of phrase
Finished
Google Speech Recognition could not understand audio
Listening ...
Starti

## noise sample 받는 거 시도

In [9]:
import pyaudio
import wave
import audioop
from collections import deque
import os
import urllib
import time
import math
import numpy as np
import noisereduce as nr
from scipy.io import wavfile


# Microphone stream config.
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
THRESHOLD = 1500
SILENCE_LIMIT = 1 
PREV_AUDIO = 0.5
RECORD_SECONDS = 10

#Open stream
p = pyaudio.PyAudio()
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

#noise window
data = stream.read(10000,exception_on_overflow=False)
noise_sample = np.frombuffer(data, dtype=np.float32)
print("Noise Sample")
loud_threshold = np.mean(np.abs(noise_sample)) * 10
print("Loud threshold", loud_threshold)
audio_buffer = []
near = 0


def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)


def listen_for_speech(threshold=loud_threshold, num_phrases=-1):    
    print("* Listening mic. ")
    audio2send = []
    cur_data = '' 
    rel = RATE/CHUNK
    slid_win = deque(maxlen=int(SILENCE_LIMIT * rel))
    prev_audio = deque(maxlen=int(PREV_AUDIO * rel))
    started = False
    n = num_phrases
    response = []

    while (num_phrases == -1 or n > 0):
        cur_data = stream.read(CHUNK,exception_on_overflow=False)
        slid_win.append(math.sqrt(abs(audioop.avg(cur_data, 4))))
        if(sum([x > threshold for x in slid_win]) > 0):
            if(not started):
                print("Starting record of phrase")
                started = True
            audio2send.append(cur_data)
        elif (started is True):
            print("Finished")
            filename = save_speech(list(prev_audio) + audio2send, p)
            started = False
            slid_win = deque(maxlen=int(SILENCE_LIMIT * rel))
            prev_audio = deque(maxlen=int(0.5 * rel)) 
            audio2send = []
            n -= 1
            print("Listening ...")
        else:
            prev_audio.append(cur_data)

    print("* Done recording")
    stream.close()
    p.terminate()

    return response


def save_speech(data, p):
    try:
        foldername = 'original_audio/'+str(time.strftime('%Y-%m-%d'))+'/'
        createFolder(foldername)
        filename = foldername+'original_output_'+str(int(time.strftime('%H%M%S')))
        data = b''.join(data)
        wf = wave.open(filename + '.wav', 'wb')
        wf.setnchannels(1)
        wf.setsampwidth(p.get_sample_size(FORMAT))
        wf.setframerate(RATE) 
        wf.writeframes(data)
        wf.close()
        return filename + '.wav'
    except data is None:
        print('Stop Recording')


if(__name__ == '__main__'):
    listen_for_speech()
    audio_int()

Noise Sample
Loud threshold nan
* Listening mic. 


/Users/jaeyoungshin/opt/anaconda3/envs/python3.7/lib/python3.7/site-packages/numpy/core/_methods.py:160: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/Users/jaeyoungshin/opt/anaconda3/envs/python3.7/lib/python3.7/site-packages/numpy/core/_methods.py:160: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)


KeyboardInterrupt: 

## 실시간 녹음파일 노이즈리덕션o + 원본

In [17]:
import pyaudio
import wave
import audioop
from collections import deque
import os
import urllib
import time
import math
import numpy as np
import noisereduce as nr
from scipy.io import wavfile

CHUNK = 1024 
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
THRESHOLD = 2000
SILENCE_LIMIT = 0.5 
PREV_AUDIO = 0.5
RECORD_SECONDS = 10
snr = 2

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

        
def audio_int(num_samples=10):
    print("Getting intensity values from mic.")
    p = pyaudio.PyAudio()

    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    values = [math.sqrt(abs(audioop.avg(stream.read(CHUNK), 4))) 
              for x in range(num_samples)] 
    values = sorted(values, reverse=True)
    r = np.mean(values)
    print(" Finished ")
    print(" Average audio intensity is ", r)
    stream.close()
    p.terminate()
    return r

tmp = []

def listen_for_speech(threshold=THRESHOLD):
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    print("* Listening mic. ")
    audio2send = []; noise2send = [0.0 for x in range(10)]
    cur_data = ''  
    rel = RATE/CHUNK
    slid_win = deque(maxlen=int(SILENCE_LIMIT * rel))
    prev_audio = deque(maxlen=int(PREV_AUDIO * rel))
    started = False
    response = []
    
    while True:
        cur_data = stream.read(CHUNK,exception_on_overflow=False)
        slid_win.append(math.sqrt(abs(audioop.avg(cur_data, 4))))
        if(sum([(x > threshold) for x in slid_win]) > 0):
            if(not started):
                print("Starting record of phrase")
                started = True
            audio2send.append(cur_data)
        elif (started is True):
            print("Finished")

            filename_nr = save_file(list(prev_audio) + audio2send, p)
            filename_or = save_speech(list(prev_audio) + audio2send, p)
            text_nr = stt_google_wav(filename_nr)
            text_or = stt_google_wav(filename_or)
            started = False
            slid_win = deque(maxlen=int(SILENCE_LIMIT * rel))
            prev_audio = deque(maxlen=int(0.5 * rel))
 #           tmp += list(prev_audio) + audio2send
            audio2send = []

            print("Listening ...")
        else:
            prev_audio.append(cur_data)

    filename = save_file(tmp, p)
    text = stt_google_wav(filename)
    print("* Done recording")
    stream.close()
    p.terminate()

    return response


def save_file(data, p):
    foldername = 'noise_reduction_audio/'+str(time.strftime('%Y-%m-%d'))+'/'
    createFolder(foldername)
    filename = foldername + 'nr_output_'+str(int(time.strftime('%H%M%S')))
    data = b''.join(data)
    wf = wave.open(filename + '.wav', 'wb')
    wf.setnchannels(1)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)  
    wf.writeframes(data)
    wf.close()
    #########
    rate, data = wavfile.read(filename + '.wav')
    data = data.astype('float32')
    noise_data = data[:]
    noise_clip = noise_data / snr
    reduced_noise = nr.reduce_noise(
        audio_clip=data, noise_clip=noise_clip, verbose=False)
    wavfile.write(filename +'.wav', rate=RATE, data=reduced_noise.astype(np.int16))
    #########
    return filename + '.wav'


def save_speech(data, p):
    try:
        foldername = 'original_audio/'+str(time.strftime('%Y-%m-%d'))+'/'
        createFolder(foldername)
        filename = foldername+'original_output_'+str(int(time.strftime('%H%M%S')))
        data = b''.join(data)
        wf = wave.open(filename + '.wav', 'wb')
        wf.setnchannels(1)
        wf.setsampwidth(p.get_sample_size(FORMAT))
        wf.setframerate(RATE) 
        wf.writeframes(data)
        wf.close()
        return filename + '.wav'
    except data is None:
        print('Stop Recording')
        

def stt_google_wav(audio_fname):
    AUDIO_FILE = f"{audio_fname}"
    text = []
    # audio file을 audio source로 사용합니다
    r = sr.Recognizer()
    with sr.AudioFile(AUDIO_FILE) as source:
        audio = r.record(source)  # 전체 audio file 읽기

    # 구글 웹 음성 API로 인식하기 (하루에 제한 50회)
    try:
        print(f"{audio_fname[:8]} STT result : " + r.recognize_google(audio, language='ko'))
    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand audio")
    except sr.RequestError as e:
        print("Could not request results from Google Speech Recognition service; {0}".format(e))

if(__name__ == '__main__'):
    listen_for_speech()
    audio_int()

* Listening mic. 
Starting record of phrase


KeyboardInterrupt: 

## noise_sample 받아서 시도 중

In [4]:
import pyaudio
import wave
import audioop
from collections import deque
import os
import urllib
import time
import math
import numpy as np
import noisereduce as nr
from scipy.io import wavfile

CHUNK = 1024 
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
THRESHOLD = 2000
SILENCE_LIMIT = 0.5 
PREV_AUDIO = 0.5
RECORD_SECONDS = 10
snr = 2


p = pyaudio.PyAudio()
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)
#noise window
# data = stream.read(10000,exception_on_overflow=False)
# noise_sample = np.frombuffer(data, dtype=np.float32)
# print("Noise Sample", noise_sample)
# loud_threshold = np.mean(np.abs(noise_sample)) * 10
# print("Loud threshold", loud_threshold)
# audio_buffer = []
# near = 0


def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

def listen_for_speech(threshold=THRESHOLD):


    print("* Listening mic. ")
    audio2send = []; noise2send = [0.0 for x in range(10)]
    rel = RATE/CHUNK
    slid_win = deque(maxlen=int(SILENCE_LIMIT * rel))
    prev_audio = deque(maxlen=int(PREV_AUDIO * rel))
    started = False
    response = []
    
    while True:
        data = stream.read(CHUNK,exception_on_overflow=False)
        current_window = np.frombuffer(data, dtype=np.float32)
        cur_data = nr.reduce_noise(audio_clip=current_window, noise_clip=noise_sample, verbose=False)

        slid_win.append(math.sqrt(abs(audioop.avg(cur_data, 4))))
        if(sum([(x > threshold) for x in slid_win]) > 0):
            if(not started):
                print("Starting record of phrase")
                started = True
            audio2send.append(cur_data)
        elif (started is True):
            print("Finished")

            filename = save_file(list(prev_audio) + audio2send, p)
            started = False
            slid_win = deque(maxlen=int(SILENCE_LIMIT * rel))
            prev_audio = deque(maxlen=int(0.5 * rel)) 
            audio2send = []

            print("Listening ...")
        else:
            prev_audio.append(cur_data)

    print("* Done recording")
    stream.close()
    p.terminate()

    return response


def save_file(data, p):
    foldername = 'noise_reduction_audio/'+str(time.strftime('%Y-%m-%d'))+'/'
    createFolder(foldername)
    filename = foldername + 'nr_output_'+str(int(time.strftime('%H%M%S')))
    data = b''.join(data)
    wf = wave.open(filename + '.wav', 'wb')
    wf.setnchannels(1)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE) 
    wf.writeframes(data)
    wf.close()
    #########
#     rate, data = wavfile.read(filename + '.wav')
#     data = data.astype('float32')
#     noise_data = data[:]
#     noise_clip = noise_data / snr
#     reduced_noise = nr.reduce_noise(
#         audio_clip=data, noise_clip=noise_clip, verbose=False)
#     wavfile.write(filename +'.wav', rate=RATE, data=reduced_noise.astype(np.int16))
#     #########
    return filename + '.wav'


if(__name__ == '__main__'):
    listen_for_speech()
    audio_int()

* Listening mic. 
Starting record of phrase


KeyboardInterrupt: 

In [2]:
import pyaudio
import wave
import audioop
from collections import deque
import os
import urllib
import time
import math
import numpy as np
import noisereduce as nr
from scipy.io import wavfile

/Users/jaeyoungshin/opt/anaconda3/envs/python3.7/lib/python3.7/site-packages/noisereduce/noisereduce.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [5]:
CHUNKSIZE = 1024 # fixed chunk size
RATE = 16000
RECORD_SECONDS = 10
FORMAT = pyaudio.paInt16
CHANNELS = 1

# initialize portaudio
p = pyaudio.PyAudio()
stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNKSIZE)

#noise window
data = stream.read(10000,exception_on_overflow=False)
noise_sample = np.fromstring(data, dtype=np.int16)
print("Noise Sample")
plotAudio2(noise_sample)
loud_threshold = np.mean(np.abs(noise_sample)) * 10
print("Loud threshold", loud_threshold)
audio_buffer = []
near = 0

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
        
def save_file(data,p):
    foldername = 'noise_reduction_audio/'+str(time.strftime('%Y-%m-%d'))+'/'
    createFolder(foldername)
    filename = foldername + 'nr_output_'+str(int(time.strftime('%H%M%S')))
    data = b''.join(data)
    wf = wave.open(filename + '.wav', 'wb')
    wf.setnchannels(1)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)  
    wf.writeframes(data)
    wf.close()
    #########
#     rate, data = wavfile.read(filename + '.wav')
#     data = data.astype('float32')
# #     noise_reduced = nr.reduce_noise(
# #         audio_clip=data, noise_clip=noise.astype('float32'), use_tensorflow=False,verbose=False)
#     wavfile.write(filename +'.wav', rate=RATE, data=noise_reduced.astype(np.int16))
#     #########
    return filename + '.wav'
try:
    while(True):
        # Read chunk and load it into numpy array.
        data = stream.read(CHUNKSIZE,exception_on_overflow=False)
        current_window = np.fromstring(data, dtype=np.int16)
        #Reduce noise real-time
      #  current_window = nr.reduce_noise(audio_clip=current_window, noise_clip=noise_sample, verbose=False)
    
        print(current_window)

        if(audio_buffer==[]):
            audio_buffer = current_window
        else:
            if(np.mean(np.abs(current_window))<loud_threshold):
                print("Inside silence reign")       
                if(near<10):
                    audio_buffer = np.concatenate((audio_buffer,current_window))
                    near += 1
                else:
                    predictSound(np.array(audio_buffer))
                    audio_buffer = []
                    near
            else:
                print("Inside loud reign")
                near = 0
                audio_buffer = np.concatenate((audio_buffer,current_window))
except:
    file = audio_buffer
    filename = save_file(file, p)  ##
        
# close stream
stream.stop_stream()
stream.close()
p.terminate()

/Users/jaeyoungshin/opt/anaconda3/envs/python3.7/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  del sys.path[0]
/Users/jaeyoungshin/opt/anaconda3/envs/python3.7/lib/python3.7/site-packages/ipykernel_launcher.py:50: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
/Users/jaeyoungshin/opt/anaconda3/envs/python3.7/lib/python3.7/site-packages/ipykernel_launcher.py:56: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


Noise Sample
Loud threshold 1209.966
[-250 -247 -222 ...   27   25   17]
[  21   19    8 ... -145 -149 -153]
Inside silence reign
[-164 -176 -186 ... -196 -195 -184]
Inside silence reign
[-165 -148 -132 ...   38   38   39]
Inside silence reign
[  30   27   25 ... -278 -290 -276]
Inside silence reign
[-251 -251 -247 ...  192  204   38]
Inside silence reign
[-11  61  92 ... 126 112  85]
Inside silence reign
[  81   60   61 ... -171  -19  -20]
Inside silence reign
[ -55  -41  -48 ... -108  -93  -94]
Inside silence reign
[ -68  -56  -47 ... -178 -173 -174]
Inside silence reign
[-172 -173 -183 ... -191 -198 -202]
Inside silence reign
[-193 -184 -173 ...   95   97   94]
Inside silence reign
[  87   80   77 ...  -87  -97 -109]
Inside silence reign
[-120 -128 -137 ...  155  150  148]
Inside silence reign
[141 137 138 ... 180 182 183]
Inside silence reign
[ 179  178  178 ... -140 -144 -144]
Inside silence reign
[-148 -162 -163 ...   55   58   86]
Inside silence reign
[ 129  169  199 ... -179   

[-50 -69 -83 ...  67  96 108]
Inside silence reign
[109 122 143 ... 176 173 170]
Inside silence reign
[162 155 147 ... -76 -72 -66]
Inside silence reign
[-59 -51 -48 ... -58 -44 -30]
Inside silence reign
[-20 -10  -1 ... 175 165 151]
Inside silence reign
[ 131  112   90 ... -104 -104 -114]
Inside silence reign
[-127 -128 -130 ...  -84  -76  -67]
Inside silence reign
[-55 -48 -35 ...  61  48  43]
Inside silence reign
[ 38  26  15 ... -92 -86 -70]
Inside silence reign
[-45 -29 -15 ... 138 143 144]
Inside silence reign
[144 150 151 ... -36 -55 -68]
Inside silence reign
[-75 -86 -86 ... -61 -53 -46]
Inside silence reign
[ -38  -30  -24 ... 1226 1420 1673]
Inside silence reign
[1935 2085 2188 ... 1390 1309 1253]
Inside loud reign
[1154 1007  819 ... -210 -239 -223]
Inside loud reign
[-173 -117  -58 ...  174  182  179]
Inside silence reign
[  184   181   162 ...   292  -243 -1010]
Inside loud reign
[-1963 -2988 -4178 ...  1409  1545  1620]
Inside loud reign
[1622 1591 1535 ...   62   31   -8